In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('Twitter_Data(Relabeled).csv')
dataset.columns=['clean_text','category']
dataset['category']=pd.to_numeric(dataset['category'],errors='coerce')
dataset=dataset.dropna()
dataset['category']=dataset['category'].astype(int)
train , test =train_test_split(dataset,train_size=0.7)
train.head(10)

,clean_text,category
75403,modis broadcast poll panel likely decide today...,0
104623,jaitely underestimated modi remember jaitely t...,2
31300,dont believe left newsbtw modi will win ✌️,2
143361,shri narendra modi addresses public meeting go...,1
59384,weaker sections have benefited largely from po...,0
110967,just because modi refuses bend down front the ...,0
98129,everything was done modi get sympathy votes fr...,2
129500,after achieving independence long time back la...,2
48441,may the lord give bear with modi and his elect...,1
27573,when modi introduced scheme inr 6000 farmers y...,0


In [2]:
test.head(10)

,clean_text,category
93196,modi good but his policies are not says congre...,2
151111,right spreading hate and spreading love peace ...,2
50245,everyone knows option left people will vote fo...,2
18642,auntie sir son hates every indian citizen modi...,1
121337,modi his zindabadwallas believe that stark tru...,0
40084,report from the ground slap the faces the sche...,1
138786,modi made everyone which ever party may belong...,2
49980,thought iaf special aircraft bringing back vij...,2
97941,remember how your media was covering pakistan ...,1
147958,well you need take positivly more unemployemen...,2


In [3]:
#Using a learning rate of 3e-5
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
print(torch.cuda.is_available())
model_args = ClassificationArgs(num_train_epochs=5, max_seq_length=128,
                                train_batch_size=8, eval_batch_size=8, learning_rate=3e-5, adam_epsilon=1e-08,
                                use_multiprocessing=False,
                                output_dir='./outputs/', overwrite_output_dir=True,
                                save_model_every_epoch=True)
model = ClassificationModel(
    'xlnet', 'xlnet-base-cased', num_labels=3, args=model_args, use_cuda=True)


True


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [4]:
model.train_model(train, use_cuda=True, verbose=True)

/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:585: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 5 of 5: 100%|██████████| 5/5 [3:17:44<00:00, 2372.95s/it]


(71300, 0.1710459726249143)

In [11]:
from sklearn.metrics import accuracy_score
results, model_outputs, wrong_predictions = model.eval_model(
    test, acc=accuracy_score)
print(results)


/home/rafxar/Proyecto-Final-De-Carrera-1/BERT/bert-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1426: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation: 100%|██████████| 6112/6112 [04:37<00:00, 22.06it/s]

{'mcc': 0.968946563736971, 'acc': 0.98009858665194, 'eval_loss': 0.11044914731885801}


In [3]:
#Using a learning rate of 5e-5
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score
import torch
print(torch.cuda.is_available())
model_args = ClassificationArgs(num_train_epochs=5, max_seq_length=128,
                                train_batch_size=8, eval_batch_size=8, learning_rate=5e-5, adam_epsilon=1e-08,
                                use_multiprocessing=False,
                                output_dir='./outputs/', overwrite_output_dir=True,
                                save_model_every_epoch=True)
model = ClassificationModel(
    'xlnet', 'xlnet-base-cased', num_labels=3, args=model_args, use_cuda=True)
model.train_model(train, use_cuda=True, verbose=True, eval_data=test,acc=accuracy_score)
results, model_outputs, wrong_predictions = model.eval_model(
    test, acc=accuracy_score)
print(results)

True


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 874851584 vs 874851472

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/rafxar/Proyecto-Final-De-Carrera-1/bert-env/lib/python3.9/site-packages/tensorboard/summary/writer/event_file_writer.py", line 247, in run
    self._record_writer.flush()
  File "/home/rafxar/Proyecto-Final-De-Carrera-1/bert-env/lib/python3.9/site-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/home/rafxar/Proyecto-Final-De-Carrera-1/bert-env/lib/python3.9/site-packages/tensorflow/python/lib/io/file_io.py", line 223, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.ResourceExhaustedError: runs/Dec19_03-25-38_rafxar-manjaro/events.out.tfevents.1639902339.rafxar-manjaro.724314.0; No space left on device


In [5]:
results, model_outputs, wrong_predictions = model.eval_model(
    test, acc=accuracy_score)
print(results)

/home/rafxar/Proyecto-Final-De-Carrera-1/bert-env/lib/python3.9/site-packages/simpletransformers/classification/classification_model.py:1426: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation: 100%|██████████| 6112/6112 [04:12<00:00, 24.24it/s]

{'mcc': 0.0, 'acc': 0.3381603976191937, 'eval_loss': 1.0796993182318686}


In [3]:
#Using a learning rate of 3e-4
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score
import torch
print(torch.cuda.is_available())
model_args = ClassificationArgs(num_train_epochs=5, max_seq_length=128,
                                train_batch_size=8, eval_batch_size=8, learning_rate=3e-4, adam_epsilon=1e-08,
                                use_multiprocessing=False,
                                output_dir='./outputs/', overwrite_output_dir=True,
                                save_model_every_epoch=True)
model = ClassificationModel(
    'xlnet', 'xlnet-base-cased', num_labels=3, args=model_args, use_cuda=True)
model.train_model(train, use_cuda=True, verbose=True, eval_data=test,acc=accuracy_score)
results, model_outputs, wrong_predictions = model.eval_model(
    test, acc=accuracy_score)
print(results)

True


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation: 100%|██████████| 6112/6112 [04:20<00:00, 23.42it/s]


{'mcc': 0.0, 'acc': 0.4434967580945368, 'eval_loss': 1.0591390120421404}
